In [1]:
import requests
import pandas as pd
import time

POLYGON_KEY = "ML8KNIhH8hbBS9Cv_w9YcHfwqEpp3IQZ"  # 🔐 Replace with your actual API key

# Step 1: Load all unique tickers from stocks.csv
df = pd.read_csv("stocks.csv", parse_dates=["date"])
symbols = sorted(df["symbol"].dropna().unique())

# Step 2: Query Polygon's ticker details
def get_market_cap(symbol):
    url = f"https://api.polygon.io/v3/reference/tickers/{symbol.upper()}"
    params = {"apiKey": POLYGON_KEY}
    try:
        res = requests.get(url, params=params)
        res.raise_for_status()
        data = res.json()
        market_cap = data.get("results", {}).get("market_cap", None)
        name = data.get("results", {}).get("name", "")
        return {"symbol": symbol, "market_cap": market_cap, "company_name": name}
    except Exception as e:
        print(f"⚠️ {symbol}: {e}")
        return {"symbol": symbol, "market_cap": None, "company_name": None}

# Step 3: Loop with rate-limiting (optional for safety)
results = []
for i, sym in enumerate(symbols, 1):
    print(f"📡 [{i}/{len(symbols)}] {sym}")
    info = get_market_cap(sym)
    results.append(info)
    time.sleep(0.5)  # ✅ adjust if needed — less aggressive with paid plan

# Step 4: Save to CSV
market_cap_df = pd.DataFrame(results)
market_cap_df = market_cap_df.dropna(subset=["market_cap"])
market_cap_df.to_csv("market_cap.csv", index=False)

print("✅ Done. Saved to market_cap.csv")


📡 [1/502] A
📡 [2/502] AAPL
📡 [3/502] ABBV
📡 [4/502] ABNB
📡 [5/502] ABT
📡 [6/502] ACGL
📡 [7/502] ACN
📡 [8/502] ADBE
📡 [9/502] ADI
📡 [10/502] ADM
📡 [11/502] ADP
📡 [12/502] ADSK
📡 [13/502] AEE
📡 [14/502] AEP
📡 [15/502] AES
📡 [16/502] AFL
📡 [17/502] AIG
📡 [18/502] AIZ
📡 [19/502] AJG
📡 [20/502] AKAM
📡 [21/502] ALB
📡 [22/502] ALGN
📡 [23/502] ALL
📡 [24/502] ALLE
📡 [25/502] AMAT
📡 [26/502] AMCR
📡 [27/502] AMD
📡 [28/502] AME
📡 [29/502] AMGN
📡 [30/502] AMP
📡 [31/502] AMT
📡 [32/502] AMZN
📡 [33/502] ANET
📡 [34/502] AON
📡 [35/502] AOS
📡 [36/502] APA
📡 [37/502] APD
📡 [38/502] APH
📡 [39/502] APO
📡 [40/502] APTV
📡 [41/502] ARE
📡 [42/502] ATO
📡 [43/502] AVB
📡 [44/502] AVGO
📡 [45/502] AVY
📡 [46/502] AWK
📡 [47/502] AXON
📡 [48/502] AXP
📡 [49/502] AZO
📡 [50/502] BA
📡 [51/502] BAC
📡 [52/502] BALL
📡 [53/502] BAX
📡 [54/502] BBY
📡 [55/502] BDX
📡 [56/502] BEN
📡 [57/502] BF.B
📡 [58/502] BG
📡 [59/502] BIIB
📡 [60/502] BK
📡 [61/502] BKNG
📡 [62/502] BKR
📡 [63/502] BLDR
📡 [64/502] BLK
📡 [65/502] BMY
📡 [66/502] BR
📡 [

In [6]:
import pandas as pd

# Load your raw market cap data
df = pd.read_csv("market_cap.csv")

# Step 1: Parse market cap strings like '3.2T', '750B', '44M'
def parse_market_cap(value):
    if pd.isna(value):
        return None
    value = str(value).upper().replace("$", "").strip()
    try:
        if value.endswith("T"):
            return float(value[:-1]) * 1e12
        elif value.endswith("B"):
            return float(value[:-1]) * 1e9
        elif value.endswith("M"):
            return float(value[:-1]) * 1e6
        else:
            return float(value)
    except:
        return None

df["cap_numeric"] = df["market_cap"].apply(parse_market_cap)

# Step 2: Assign cap score (1 = mega, 5 = small)
def get_cap_score(cap):
    if pd.isna(cap):
        return 5  # treat unknowns as smallest
    elif cap >= 200e9:
        return 1
    elif cap >= 50e9:
        return 2
    elif cap >= 10e9:
        return 3
    elif cap >= 2e9:
        return 4
    else:
        return 5

df["cap_score"] = df["cap_numeric"].apply(get_cap_score)

# Step 3: Optional emoji mapping (for Streamlit)
emoji_map = {1: "🏆", 2: "💎", 3: "🔷", 4: "🟢", 5: "🟡"}
df["cap_emoji"] = df["cap_score"].map(emoji_map)

# Step 4: Save cleaned version
output_cols = ["symbol", "company_name", "market_cap", "cap_numeric", "cap_score", "cap_emoji"]
df[output_cols].to_csv("market_cap.csv", index=False)

print("✅ Updated market_cap.csv with cap_score and cap_emoji columns.")


✅ Updated market_cap.csv with cap_score and cap_emoji columns.
